In [1]:
import os
from tqdm import tqdm

import config
import modules.classification_dataloaders as classification_data_loader
import modules.dataloaders as detection_data_loader
import modules.metrics as metrics

import numpy as np
import torch
import torchmetrics
from torchmetrics.detection.mean_ap import MeanAveragePrecision

In [2]:
import onnx
import onnxruntime

# Check Models

## Classifier Medium Compression: 25,59 KB

In [3]:
classifier = onnx.load('./onnx_models/medium_fassd__conv341_big__epoch=93.onnx')
onnx.checker.check_model(classifier)

## Detector with Compression and w8a8b8

In [4]:
detector = onnx.load('./onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx')
onnx.checker.check_model(detector)

# Classification Loaders

In [5]:
#classification_loader = classification_data_loader.get_val_loader(shuffle=False)


TEST DFire dataset
DFire Removed wrong images: 0
DFire empty images: 2005
DFire only smoke images: 1186
DFire only fire images: 220
DFire smoke and fire images: 895

Test dataset len: 4306

TEST FASDD UAV dataset
DFire Removed wrong images: 0
DFire empty images: 1997
DFire only smoke images: 846
DFire only fire images: 35
DFire smoke and fire images: 1303

Test FASDD UAV dataset len: 4181

TEST FASDD CV dataset
DFire Removed wrong images: 0
DFire empty images: 6533
DFire only smoke images: 3902
DFire only fire images: 2091
DFire smoke and fire images: 3358

Test FASDD CV dataset len: 15884

Concatenate Test DFire and FASDD UAV datasets
Test dataset len: 8487
Concatenate with FASDD CV dataset
Test dataset len: 24371


# Helper function to convert pytorch tensors to numpy. Useful to handle datatset output.

In [5]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# Evaluate ONNX with F1 Mean, Smoke and Fire

In [7]:
# precision_metric = torchmetrics.classification.MultilabelPrecision(num_labels = config.N_CLASSES, 
#                                                                    threshold = 0.5, 
#                                                                    average = None).to('cpu')
# recall_metric = torchmetrics.classification.MultilabelRecall(num_labels = config.N_CLASSES, 
#                                                              threshold = 0.5, 
#                                                              average = None).to('cpu')
# accuracy_metric = torchmetrics.classification.MultilabelAccuracy(num_labels = config.N_CLASSES, 
#                                                                  threshold = 0.5, 
#                                                                  average = None).to('cpu')
# f1_metric = torchmetrics.classification.MultilabelF1Score(num_labels = config.N_CLASSES, 
#                                                           threshold = 0.5, 
#                                                           average = None).to('cpu')

# f1_metric_mean = torchmetrics.classification.MultilabelF1Score(num_labels = config.N_CLASSES, 
#                                                                threshold = 0.5, 
#                                                                average = 'macro').to('cpu')

### Classifier evaluation funtion

Out of ONNX Model:
- list with 1 np.array
- [ np.array(batch, shape of 1 inference) ]
- to access 1 inference: out[0][0]

In [8]:
# def eval_classifier_onnx(loader, model_name):

#     ort_session = onnxruntime.InferenceSession(model_name, providers=["CPUExecutionProvider"])

#     precision_metric.reset()
#     recall_metric.reset()
#     accuracy_metric.reset()
#     f1_metric.reset()
#     f1_metric_mean.reset()
    
#     loop = tqdm(loader, desc='Validating', leave=True)

#     for batch_idx, (img, label) in enumerate(loop):

#         for idx in range(config.BATCH_SIZE):
            
#             ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img[idx].unsqueeze(dim=0))}
#             yhat = ort_session.run(None, ort_inputs)
#             yhat = np.array(yhat)
#             #yhat = torch.tensor(yhat).squeeze(dim=0)
#             yhat = torch.sigmoid(torch.tensor(yhat).squeeze(dim=0))
#             target = label[idx].unsqueeze(dim=0)

#             precision_metric.update(yhat, target)
#             recall_metric.update(yhat, target)
#             accuracy_metric.update(yhat, target)
#             f1_metric.update(yhat, target)
#             f1_metric_mean.update(yhat, target)
    
#     precision = precision_metric.compute()
#     recall = recall_metric.compute()
#     accuracy = accuracy_metric.compute()
#     f1 = f1_metric.compute()
#     f1_mean = f1_metric_mean.compute()

#     precision_metric.reset()
#     recall_metric.reset()
#     accuracy_metric.reset()
#     f1_metric.reset()
#     f1_metric_mean.reset()

#     print(f'SMOKE -> Precision: {precision[0]:.4f} - Recall: {recall[0]:.4f} - Accuracy: {accuracy[0]:.4f} - F1: {f1[0]:.4f}')
#     print(f'FIRE -> Precision: {precision[1]:.4f} - Recall: {recall[1]:.4f} - Accuracy: {accuracy[1]:.4f} - F1: {f1[1]:.4f}')
#     print(f'Mean F1 Score: {f1_mean.item():.4f}')
    
#     return (
#         {
#         'Accuracy': [accuracy[0].item(), accuracy[1].item()],
#         'Precision': [precision[0].item(), precision[1].item()],
#         'Recall': [recall[0].item(), recall[1].item()],
#         'F1': [f1[0].item(), f1[1].item()],
#         'F1 mean': f1_mean.item(),
#         }
#     )

# Evaluate Classifier Model

In [9]:
# print("\n________________________________ Classifier MEDIUM COMPRESSION _______________________________")
# _ = eval_classifier_onnx(classification_loader, './onnx_models/medium_fassd__conv341_big__epoch=93.onnx')


________________________________ Classifier MEDIUM COMPRESSION _______________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [01:12<00:00,  5.21it/s]

SMOKE -> Precision: 0.9099 - Recall: 0.8943 - Accuracy: 0.9084 - F1: 0.9020
FIRE -> Precision: 0.9032 - Recall: 0.9697 - Accuracy: 0.9565 - F1: 0.9353
Mean F1 Score: 0.9187


### Results with Full Dataset

- SMOKE -> Precision: 0.9099 - Recall: 0.8943 - Accuracy: 0.9084 - F1: 0.9020
- FIRE -> Precision: 0.9032 - Recall: 0.9697 - Accuracy: 0.9565 - F1: 0.9353
- Mean F1 Score: 0.9187

# Detection Loaders

In [6]:
detection_loader = detection_data_loader.get_val_loader()


TEST DFire dataset
DFire Removed wrong images: 0
DFire Removed due to overlapping: 310
DFire Removed due to more than 10: 13

Test DFire dataset len: 3983

TEST FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 377
FASDD Removed due to more than 10: 156

Test FASDD UAV dataset len: 3648

TEST FASDD CV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 317
FASDD Removed due to more than 10: 44

Test FASDD CV dataset len: 15523

Concatenate Test DFire and FASDD UAV datasets
Test dataset len: 7631
Concatenate with FASDD CV dataset
Test dataset len: 23154


# Evaluate Detector

In [7]:
map_metric = MeanAveragePrecision(
    box_format='xyxy',
    iou_thresholds=[config.IOU_THRESHOLD],
    class_metrics=True, # Enables separated metrics for each class
    #average='micro',
    extended_summary=False).to('cpu')

### Score Threshold

In [8]:
SCORE_THRES = 0.2

### Evaluation Function

In [9]:
def eval_detector_onnx(loader, model_name, score_thres):

    ort_session = onnxruntime.InferenceSession(model_name, providers=["CPUExecutionProvider"])

    map_metric.reset()
    
    loop = tqdm(loader, desc='Validating', leave=True)

    for batch_idx, (img, label) in enumerate(loop):

        for idx in range(config.BATCH_SIZE):
            
            ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img[idx].unsqueeze(dim=0))}
            out = ort_session.run(None, ort_inputs)
            
            # out of onnx session is a list: [out_tensor with batch dim] -> [ (1,12,7,7) ] 
            # -> out[0] = (1,12,7,7)
            # -> out[0][0] = (12,7,7)
            #print(f'Out type: {type(out[0])} - Output shape: {out[0].shape}')
            out = torch.tensor(np.array(out[0]))
            out = out.permute(0, 2, 3, 1)
            #print(f'Out shape after permute: {out.shape}')
            #print(f'Out shape after indexing: {out[0].shape}')
            
            # Label should be [xc, yc, w, h, score=1, smoke, fire] in 7x7 square
            #print(f'Label indexed shape: {label[idx].shape}')
            
        # Mean Average Precision
            target_boxes = metrics.get_true_boxes(label[idx].detach().to('cpu'))
            pred_boxes = metrics.get_pred_boxes(
                model_out = out[0].detach().to('cpu'),
                score_threshold=score_thres)
            map_metric.update(preds = pred_boxes, target = target_boxes)
    
    meanAP = map_metric.compute()
    map_metric.reset()

    print(f'Smoke -> AP: {meanAP["map_per_class"][0].item():.4f} - AR: {meanAP["mar_100_per_class"][0].item():.4f}')
    print(f'Fire -> AP: {meanAP["map_per_class"][1].item():.4f} - AR: {meanAP["mar_100_per_class"][1].item():.4f}')
    print(f'mAP: {meanAP["map_50"].item():.4f}')
    
    return (
        {'mAP': meanAP['map_50'].item(),
         'AP': [meanAP['map_per_class'][0].item(), meanAP['map_per_class'][1].item()],
         'AR': [meanAP['mar_100_per_class'][0].item(), meanAP['mar_100_per_class'][1].item()]
        }
    )

In [10]:
print("\n________________________________ Detector MEDIUM COMPRESSION _______________________________")
_ = eval_detector_onnx(
    detection_loader, 
    './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx',
    SCORE_THRES
)


________________________________ Detector MEDIUM COMPRESSION _______________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [01:18<00:00,  4.58it/s]


Smoke -> AP: 0.6470 - AR: 0.7070
Fire -> AP: 0.6108 - AR: 0.6680
mAP: 0.6289


# Classify 1st + Detection 2nd

In [11]:
def eval_classifier_plus_detector_onnx(
    loader, 
    classifier_model_name,
    detector_model_name,
    classification_thres=0,
    score_thres=0.2
):

    classify_session = onnxruntime.InferenceSession(classifier_model_name, providers=["CPUExecutionProvider"])
    detect_session = onnxruntime.InferenceSession(detector_model_name, providers=["CPUExecutionProvider"])

    map_metric.reset()
    
    loop = tqdm(loader, desc='Validating', leave=True)

    for batch_idx, (img, label) in enumerate(loop):

        for idx in range(config.BATCH_SIZE):
            
            classify_inputs = {classify_session.get_inputs()[0].name: to_numpy(img[idx].unsqueeze(dim=0))}
            classification_out = classify_session.run(None, classify_inputs)
            # print(f'Smoke pred: {classification_out[0][0][0]}')
            # print(f'Fire pred: {classification_out[0][0][1]}')
            
            # Use Detector if Classifier predicts fire or smoke
            if ( classification_out[0][0][0] >= classification_thres 
                or 
                classification_out[0][0][1] >= classification_thres ):
                detect_inputs = {detect_session.get_inputs()[0].name: to_numpy(img[idx].unsqueeze(dim=0))}
                out = detect_session.run(None, detect_inputs)
                out = torch.tensor(np.array(out[0]))
                out = out.permute(0, 2, 3, 1)
            else:
                out = torch.zeros(1,7,7,12)           
            
            
        # Mean Average Precision
            target_boxes = metrics.get_true_boxes(label[idx].detach().to('cpu'))
            pred_boxes = metrics.get_pred_boxes(
                model_out = out[0].detach().to('cpu'),
                score_threshold=score_thres)
            map_metric.update(preds = pred_boxes, target = target_boxes)
    
    meanAP = map_metric.compute()
    map_metric.reset()

    print(f'Smoke -> AP: {meanAP["map_per_class"][0].item():.4f} - AR: {meanAP["mar_100_per_class"][0].item():.4f}')
    print(f'Fire -> AP: {meanAP["map_per_class"][1].item():.4f} - AR: {meanAP["mar_100_per_class"][1].item():.4f}')
    print(f'mAP: {meanAP["map_50"]:.4f}')
    
    return (
        {'mAP': meanAP['map_50'].item(),
         'AP': [meanAP['map_per_class'][0].item(), meanAP['map_per_class'][1].item()],
         'AR': [meanAP['mar_100_per_class'][0].item(), meanAP['mar_100_per_class'][1].item()]
        }
    )

### Classification Threshold = 0, Score Threshold = 0.2

Baseline Model

In [12]:
eval_classifier_plus_detector_onnx(
    loader = detection_loader,
    classifier_model_name = './onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
    detector_model_name = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx',
    classification_thres = 0,
    score_thres = 0.2)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [02:56<00:00,  2.05it/s]


Smoke -> AP: 0.6322 - AR: 0.6851
Fire -> AP: 0.6139 - AR: 0.6639
mAP: 0.6231


{'mAP': 0.6230778694152832,
 'AP': [0.6322451233863831, 0.6139106154441833],
 'AR': [0.6851410865783691, 0.6639198660850525]}

## Classification Threshold

$$
\begin{align}
\sigma(z) = \frac{1}{1-e^{-z}} \; & \Longrightarrow \; z=\ln(\sigma(z)) - \ln(1-\sigma(z)) \\
\\
\sigma(z) = 0.45 \; & \longrightarrow \; z=-0.20067 \\
\sigma(z) = 0.40 \; & \longrightarrow \; z=-0.4054651081 \\
\sigma(z) = 0.35 \; & \longrightarrow \; z=-0.6190392084 \\
\sigma(z) = 0.30 \; & \longrightarrow \; z=-0.8472978604 \\
\sigma(z) = 0.25 \; & \longrightarrow \; z=-1.098612289
\end{align}
$$

### Classification Threshold (45 %)= -0.20067, Score Threshold = 0.2

In [13]:
eval_classifier_plus_detector_onnx(
    loader = detection_loader,
    classifier_model_name = './onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
    detector_model_name = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx',
    classification_thres = -0.20067,
    score_thres = 0.2)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [03:03<00:00,  1.97it/s]


Smoke -> AP: 0.6332 - AR: 0.6899
Fire -> AP: 0.6139 - AR: 0.6644
mAP: 0.6235


{'mAP': 0.6235257387161255,
 'AP': [0.633199155330658, 0.613852322101593],
 'AR': [0.6899014115333557, 0.6643880605697632]}

### Classification Threshold (40 %) = -0.4054651081, Score Threshold = 0.2

In [14]:
eval_classifier_plus_detector_onnx(
    loader = detection_loader,
    classifier_model_name = './onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
    detector_model_name = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx',
    classification_thres = -0.4054651081,
    score_thres = 0.2)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [03:01<00:00,  1.99it/s]


Smoke -> AP: 0.6414 - AR: 0.6947
Fire -> AP: 0.6138 - AR: 0.6648
mAP: 0.6276


{'mAP': 0.627581775188446,
 'AP': [0.6413736343383789, 0.613789975643158],
 'AR': [0.6947466731071472, 0.6647626161575317]}

### Classification Threshold (35 %) = -0.6190392084, Score Threshold = 0.2

In [15]:
eval_classifier_plus_detector_onnx(
    loader = detection_loader,
    classifier_model_name = './onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
    detector_model_name = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx',
    classification_thres = -0.6190392084,
    score_thres = 0.2)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [02:59<00:00,  2.02it/s]


Smoke -> AP: 0.6421 - AR: 0.6991
Fire -> AP: 0.6136 - AR: 0.6649
mAP: 0.6279


{'mAP': 0.6278664469718933,
 'AP': [0.6421014666557312, 0.6136313676834106],
 'AR': [0.6990819573402405, 0.6648562550544739]}

### Classification Threshold (30 %) = -0.8472978604, Score Threshold = 0.2

In [17]:
eval_classifier_plus_detector_onnx(
    loader = detection_loader,
    classifier_model_name = './onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
    detector_model_name = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx',
    classification_thres = -0.8472978604,
    score_thres = 0.2)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [03:02<00:00,  1.98it/s]


Smoke -> AP: 0.6495 - AR: 0.7014
Fire -> AP: 0.6134 - AR: 0.6649
mAP: 0.6315


{'mAP': 0.6314582228660583,
 'AP': [0.6495007872581482, 0.6134156584739685],
 'AR': [0.7013770937919617, 0.664949893951416]}

### Classification Threshold (25 %) = -1.098612289, Score Threshold = 0.2

In [18]:
eval_classifier_plus_detector_onnx(
    loader = detection_loader,
    classifier_model_name = './onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
    detector_model_name = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx',
    classification_thres = -1.098612289,
    score_thres = 0.2)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [03:05<00:00,  1.95it/s]


Smoke -> AP: 0.6497 - AR: 0.7031
Fire -> AP: 0.6133 - AR: 0.6655
mAP: 0.6315


{'mAP': 0.6315064430236816,
 'AP': [0.6496885418891907, 0.6133243441581726],
 'AR': [0.7030771970748901, 0.6655117273330688]}

### Classification Threshold = 0, Score Threshold = 0.15

In [19]:
eval_classifier_plus_detector_onnx(
    loader = detection_loader,
    classifier_model_name = './onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
    detector_model_name = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx',
    score_thres = 0.15)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [03:00<00:00,  2.00it/s]


Smoke -> AP: 0.6393 - AR: 0.6993
Fire -> AP: 0.6139 - AR: 0.6699
mAP: 0.6266


{'mAP': 0.6266161799430847,
 'AP': [0.6393217444419861, 0.6139106154441833],
 'AR': [0.6993369460105896, 0.6699129343032837]}

### Classification Threshold = 0, Score Threshold = 0.1

In [20]:
eval_classifier_plus_detector_onnx(
    loader = detection_loader,
    classifier_model_name = './onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
    detector_model_name = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx',
    score_thres = 0.1)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [03:00<00:00,  2.00it/s]


Smoke -> AP: 0.6523 - AR: 0.7153
Fire -> AP: 0.6201 - AR: 0.6758
mAP: 0.6362


{'mAP': 0.6361923813819885,
 'AP': [0.6522578001022339, 0.6201269626617432],
 'AR': [0.7153179049491882, 0.6758123636245728]}

### Classification Threshold = 0, Score Threshold = 0.01

In [21]:
eval_classifier_plus_detector_onnx(
    loader = detection_loader,
    classifier_model_name = './onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
    detector_model_name = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx',
    score_thres = 0.01)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [03:38<00:00,  1.65it/s]


Smoke -> AP: 0.6740 - AR: 0.7727
Fire -> AP: 0.6251 - AR: 0.6896
mAP: 0.6496


{'mAP': 0.649551510810852,
 'AP': [0.6739742755889893, 0.6251287460327148],
 'AR': [0.7726963758468628, 0.6895776987075806]}

### Score Threshold = 0.001

In [22]:
eval_classifier_plus_detector_onnx(
    loader = detection_loader,
    classifier_model_name = './onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
    detector_model_name = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx',
    score_thres = 0.001)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [10:26<00:00,  1.73s/it]


Smoke -> AP: 0.6746 - AR: 0.7843
Fire -> AP: 0.6269 - AR: 0.6904
mAP: 0.6508


{'mAP': 0.6507548093795776,
 'AP': [0.6746267676353455, 0.626882791519165],
 'AR': [0.7843420505523682, 0.6904204487800598]}

# Aladdin Metrics to Get Precision and Precision-Recall Curves

In [23]:
import modules.metrics_mAP_aladdin as aladdin_metrics

In [24]:
''' ============================
    Cell to Box Mask
============================ '''
cell2box_mask = torch.zeros((config.S, config.S, 2))
for i in range(config.S):
    for j in range(config.S):
        cell2box_mask[i,j,0] = j
        cell2box_mask[i,j,1] = i  

### Aladdin mAP modified for ONNX, classify + detect, with classification and detection thresholds

In [25]:
def aladdin_get_bboxes(
    loader,
    classifier_model_name,
    detector_model_name,
    S=config.S,
    B=config.B,
    C=config.C,
    mask=cell2box_mask,
    device='cpu',
    iou_threshold=0.5,
    classification_thres = 0,
    score_thres=0.2,
    box_format="midpoint"):
    
    '''
    
    Return:
        - all_pred_boxes
        - all_true_boxes
        Format: [train_idx, class_prediction, prob_score, x1, y1, x2, y2]
    '''
    
    classify_session = onnxruntime.InferenceSession(classifier_model_name, providers=["CPUExecutionProvider"])
    detect_session = onnxruntime.InferenceSession(detector_model_name, providers=["CPUExecutionProvider"])
    
    
    all_pred_boxes = []
    all_true_boxes = []

    # Original Code
    # make sure model is in eval before get bboxes
    #model.eval()
    train_idx = 0

    loop = tqdm(loader, desc='Get Boxes', leave=True)
    for batch_idx, (imgs, labels) in enumerate(loop):
        # Original Code
        # Inference in GPU. Move tensor to CPU in outcell_2_outboxes
        # imgs = imgs.to(device)
        # labels = labels.to(device)


        for idx in range(config.BATCH_SIZE):
            
            classify_inputs = {classify_session.get_inputs()[0].name: to_numpy(imgs[idx].unsqueeze(dim=0))}
            classification_out = classify_session.run(None, classify_inputs)
            # print(f'Smoke pred: {classification_out[0][0][0]}')
            # print(f'Fire pred: {classification_out[0][0][1]}')
            
            # Use Detector if Classifier predicts fire or smoke
            if ( classification_out[0][0][0] >= classification_thres 
                or 
                classification_out[0][0][1] >= classification_thres ):
                detect_inputs = {detect_session.get_inputs()[0].name: to_numpy(imgs[idx].unsqueeze(dim=0))}
                out = detect_session.run(None, detect_inputs)
                out = torch.tensor(np.array(out[0]))
                out = out.permute(0, 2, 3, 1)
            else:
                out = torch.zeros(1,7,7,12)           
            
            # Original Code
            # with torch.no_grad():
            #     predictions = model(imgs)

            # Original Code
            # Remove Permute from the model
            #predictions = predictions.permute(0, 2, 3, 1) # Original Code

            #batch_size = imgs.shape[0] # Original Code

            true_bboxes = aladdin_metrics.outcell_2_outboxes(
                out_cells=labels[idx].unsqueeze(dim=0), 
                S=S, B=B, C=C, 
                mask=mask, 
                device='cpu', # Changed to cpu
                is_pred=False)
            bboxes = aladdin_metrics.outcell_2_outboxes(
                out_cells=out, 
                S=S, B=B, C=C, 
                mask=mask, 
                device='cpu', # Changed to cpu
                is_pred=True)

            for idx in range(1): # Only 1 image every time, due to ONNX prediction in CPU
                nms_boxes = aladdin_metrics.nms_yv1_getBBoxes(
                    bboxes[idx],
                    iou_threshold=iou_threshold,
                    threshold=score_thres,
                    box_format=box_format, # Midpoint, to use iou_tensor inside
                )


                # Plot some examples
                #if batch_idx == 0 and idx == 0:
                #    plot_image(x[idx].permute(1,2,0).to("cpu"), nms_boxes)
                #    print(nms_boxes)

                for nms_box in nms_boxes:
                    all_pred_boxes.append([train_idx] + nms_box)

                for box in true_bboxes[idx]:
                    # many will get converted to 0 pred, as bboxes have Conf = 1 and the rest are 0
                    if box[1] > score_thres:
                        all_true_boxes.append([train_idx] + box)

                train_idx += 1

    #model.train()
    return all_pred_boxes, all_true_boxes

# Function to Print mAP metrics

In [37]:
def print_metrics(mAP_metrics):
    mAP, avg_prec, cls_prec, cls_rec = mAP_metrics
    
    mAP_str = "mAP @0.50"
    smoke = "Smoke"
    fire = "Fire"
    
    print(f'{mAP_str:<12}' + f'{mAP:.4f}')
    print('Average Precision')
    print(f'- {smoke:<10}' + f'{avg_prec[0]:.4f}')
    print(f'- {fire:<10}' + f'{avg_prec[1]:.4f}')
    print('Class Precision')
    print(f'- {smoke:<10}' + f'{cls_prec[0]:.4f}')
    print(f'- {fire:<10}' + f'{cls_prec[1]:.4f}')  
    print('Class Recall')
    print(f'- {smoke:<10}' + f'{cls_rec[0]:.4f}')
    print(f'- {fire:<10}' + f'{cls_rec[1]:.4f}')
    print('Class F1-Score')
    smoke_f1 = 2 * (cls_prec[0] * cls_rec[0]) / (cls_prec[0] + cls_rec[0])
    fire_f1 = 2 * (cls_prec[1] * cls_rec[1]) / (cls_prec[1] + cls_rec[1])
    print(f'- {smoke:<10}' + f'{smoke_f1:.4f}')
    print(f'- {fire:<10}' + f'{fire_f1:.4f}')

### Log Path

In [26]:
log_path = './mAP_&_pr_curves/classify_&_detect/'

## Score Thres = 0.2

In [28]:
pred_boxes_1, true_boxes_1 = aladdin_get_bboxes(
    detection_loader,
    classifier_model_name = './onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
    detector_model_name = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx',
)

Get Boxes: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [03:15<00:00,  1.85it/s]


In [ ]:
score_02_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes_1,
    true_boxes=true_boxes_1)

In [38]:
print_metrics(score_02_metrics)

mAP @0.50   0.6239
Average Precision
- Smoke     0.6343
- Fire      0.6134
Class Precision
- Smoke     0.7291
- Fire      0.6776
Class Recall
- Smoke     0.6855
- Fire      0.6640
Class F1-Score
- Smoke     0.7066
- Fire      0.6707


## Score Thres = 0.001

In [33]:
pred_boxes_2, true_boxes_2 = aladdin_get_bboxes(
    detection_loader,
    classifier_model_name = './onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
    detector_model_name = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx',
    score_thres = 0.001
)

Get Boxes: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [10:42<00:00,  1.78s/it]


In [ ]:
score_0001_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_1e-3/',
    pred_boxes=pred_boxes_2,
    true_boxes=true_boxes_2)

In [39]:
print_metrics(score_0001_metrics)

mAP @0.50   0.6519
Average Precision
- Smoke     0.6769
- Fire      0.6270
Class Precision
- Smoke     0.0254
- Fire      0.1187
Class Recall
- Smoke     0.7846
- Fire      0.6905
Class F1-Score
- Smoke     0.0492
- Fire      0.2026


## Classification Threshold (30 %) =  Score Thres = 0.2

In [35]:
pred_boxes_3, true_boxes_3 = aladdin_get_bboxes(
    detection_loader,
    classifier_model_name = './onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
    detector_model_name = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx',
    classification_thres = -0.8472978604,
    score_thres = 0.2
)

Get Boxes: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [04:32<00:00,  1.33it/s]


In [ ]:
class_030_score_02_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'classify_030_score_thres_2e-1/',
    pred_boxes=pred_boxes_3,
    true_boxes=true_boxes_3)

In [40]:
print_metrics(class_030_score_02_metrics)

mAP @0.50   0.6314
Average Precision
- Smoke     0.6491
- Fire      0.6136
Class Precision
- Smoke     0.7234
- Fire      0.6665
Class Recall
- Smoke     0.7017
- Fire      0.6650
Class F1-Score
- Smoke     0.7124
- Fire      0.6658


## Classification Threshold (30 %) =  Score Thres = 0.25

In [41]:
pred_boxes_4, true_boxes_4 = aladdin_get_bboxes(
    detection_loader,
    classifier_model_name = './onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
    detector_model_name = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx',
    classification_thres = -0.8472978604,
    score_thres = 0.25
)

Get Boxes: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [04:31<00:00,  1.33it/s]


In [42]:
class_030_score_025_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'classify_030_score_thres_25e-2/',
    pred_boxes=pred_boxes_4,
    true_boxes=true_boxes_4)

mAP:@.5: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:10<00:00,  5.25s/it]


In [43]:
print_metrics(class_030_score_025_metrics)

mAP @0.50   0.6251
Average Precision
- Smoke     0.6402
- Fire      0.6100
Class Precision
- Smoke     0.7592
- Fire      0.7067
Class Recall
- Smoke     0.6897
- Fire      0.6597
Class F1-Score
- Smoke     0.7228
- Fire      0.6824


## Classification Threshold (30 %) =  Score Thres = 0.3

In [44]:
pred_boxes_5, true_boxes_5 = aladdin_get_bboxes(
    detection_loader,
    classifier_model_name = './onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
    detector_model_name = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx',
    classification_thres = -0.8472978604,
    score_thres = 0.3
)

Get Boxes: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [04:29<00:00,  1.34it/s]


In [45]:
class_030_score_03_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'classify_030_score_thres_3e-1/',
    pred_boxes=pred_boxes_5,
    true_boxes=true_boxes_5)

mAP:@.5: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:10<00:00,  5.02s/it]


In [46]:
print_metrics(class_030_score_03_metrics)

mAP @0.50   0.6181
Average Precision
- Smoke     0.6304
- Fire      0.6058
Class Precision
- Smoke     0.7891
- Fire      0.7457
Class Recall
- Smoke     0.6772
- Fire      0.6540
Class F1-Score
- Smoke     0.7289
- Fire      0.6968


## Classification Threshold (30 %) =  Score Thres = 0.35

In [47]:
pred_boxes_6, true_boxes_6 = aladdin_get_bboxes(
    detection_loader,
    classifier_model_name = './onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
    detector_model_name = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx',
    classification_thres = -0.8472978604,
    score_thres = 0.35
)

Get Boxes: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [04:28<00:00,  1.34it/s]


In [48]:
class_030_score_035_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'classify_030_score_thres_35e-2/',
    pred_boxes=pred_boxes_6,
    true_boxes=true_boxes_6)

mAP:@.5: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:09<00:00,  4.65s/it]


In [49]:
print_metrics(class_030_score_035_metrics)

mAP @0.50   0.6090
Average Precision
- Smoke     0.6187
- Fire      0.5993
Class Precision
- Smoke     0.8132
- Fire      0.7763
Class Recall
- Smoke     0.6626
- Fire      0.6454
Class F1-Score
- Smoke     0.7302
- Fire      0.7048


## Classification Threshold (30 %) =  Score Thres = 0.4

In [50]:
pred_boxes_7, true_boxes_7 = aladdin_get_bboxes(
    detection_loader,
    classifier_model_name = './onnx_models/medium_fassd__conv341_big__epoch=93.onnx',
    detector_model_name = './onnx_models/w8a8b8__bed_detector___aimet__fixed_point__qcdq__CPU.onnx',
    classification_thres = -0.8472978604,
    score_thres = 0.4
)

Get Boxes: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 361/361 [04:24<00:00,  1.37it/s]


In [51]:
class_030_score_04_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'classify_030_score_thres_4e-1/',
    pred_boxes=pred_boxes_7,
    true_boxes=true_boxes_7)

mAP:@.5: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:08<00:00,  4.35s/it]


In [52]:
print_metrics(class_030_score_04_metrics)

mAP @0.50   0.5977
Average Precision
- Smoke     0.6027
- Fire      0.5927
Class Precision
- Smoke     0.8366
- Fire      0.8045
Class Recall
- Smoke     0.6432
- Fire      0.6370
Class F1-Score
- Smoke     0.7273
- Fire      0.7111
